<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/RAG/advanced_rag/hybrid_and_rerank_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid and rerank

## Google Colab preparation[optional]
This is an optional step, if you want to run this notebook on Google Colab.

In [ ]:
! git clone https://github.com/milvus-io/bootcamp.git

In [ ]:
import shutil
src_dir = "./bootcamp/bootcamp/RAG/advanced_rag/rag_utils"
dst_dir = "./rag_utils"
shutil.copytree(src_dir, dst_dir)

In [ ]:
! pip install --upgrade langchain langchain-community langchain_milvus langchain-voyageai langchain-openai rank_bm25 bs4

Please prepare you [VOYAGE_API_KEY](https://docs.voyageai.com/docs/api-key-and-installation#authentication-with-api-keys) and [OPENAI_API_KEY](https://openai.com/index/openai-api/) in your environment variables.
![](imgs/colab_api_key2.png)

### If you are running this notebook on Google Colab, you have to restart this session by `Cmd/Ctrl + M`, then press `.` to make the environment take effect.

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['VOYAGE_API_KEY'] = userdata.get('VOYAGE_API_KEY')

----
## Get started


![](imgs/hybrid_and_rerank.png)

## Prepare the data

We use the Langchain WebBaseLoader to load documents from [blog sources](https://lilianweng.github.io/posts/2023-06-23-agent/) and split them into chunks using the RecursiveCharacterTextSplitter.

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from rag_utils.vanilla import vectorstore

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

## Build the chain

We load the docs into milvus vectorstore, and build a milvus retriever.

In [2]:
vectorstore.add_documents(docs)
milvus_retriever = vectorstore.as_retriever()

And build a bm25 retriever from the docs.

In [3]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(docs)

Build a vanilla RAG chain.

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.vanilla import format_docs, rag_prompt, llm


vanilla_rag_chain = (
    {"context": milvus_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Prepare hybrid_and_rerank_retriever.

In [5]:
from langchain_voyageai import VoyageAIRerank
from rag_utils.hybrid_and_rerank import RerankerRunnable

reranker = RerankerRunnable(
    compressor=VoyageAIRerank(model="rerank-lite-1"),
    top_k=4,
)

hybrid_and_rerank_retriever = {
    "milvus_retrieved_doc": milvus_retriever,
    "bm25_retrieved_doc": bm25_retriever,
    "query": RunnablePassthrough(),
} | reranker

Build hybrid_and_rerank_chain.

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

hybrid_and_rerank_chain = (
    {
        "context": hybrid_and_rerank_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Test the chain

In [7]:
query = "Which model use tools?"

vanilla_result = vanilla_rag_chain.invoke(query)
hybrid_and_rerank_result = hybrid_and_rerank_chain.invoke(query)
print(
    f"\n[vanilla_result]:\n{vanilla_result}\n\n[hybrid_and_rerank_result]:\n{hybrid_and_rerank_result}"
)

len(milvus_retrieved_doc) = 4
len(bm25_retrieved_doc) = 4
len(unique_documents) = 6

[vanilla_result]:
The models that use tools are TALM (Tool Augmented Language Models; Parisi et al. 2022), Toolformer (Schick et al. 2023), and MRKL (Modular Reasoning, Knowledge and Language; Karpas et al. 2022). These models are fine-tuned to learn to use external tool APIs and other external modules for additional capabilities.

[hybrid_and_rerank_result]:
The models that use tools are ChatGPT Plugins and OpenAI API for function calling, HuggingGPT, TALM (Tool Augmented Language Models), Toolformer, and MRKL (Modular Reasoning, Knowledge and Language). These models are equipped with the capability to use external tools or APIs to extend their functionalities.


In [hybrid_and_rerank_result], it answers with more ground truth:
> ChatGPT with Plugins and OpenAI API function calling, HuggingGPT

which does not appear in [vanilla result].

In [8]:
# hybrid_and_rerank_retriever.invoke(query)

![](imgs/hybrid_and_rerank_res1.png)

In [11]:
# milvus_retriever.invoke(query)

![](imgs/hybrid_and_rerank_res2.png)

We can see that the hybrid retrieved results include `ChatGPT` and `HuggingGPT`. This document contains the word `tool`, which is consistent with the word in the query, that is where BM25 did better.

Therefore, we can use bm25 as a supplement to vector retriever to improve the accuracy of recall.